In [1]:
!pip install vedo trimesh open3d

  Using cached vedo-2025.5.4-py3-none-any.whl.metadata (14 kB)
  Using cached trimesh-4.8.3-py3-none-any.whl.metadata (18 kB)
  Using cached open3d-0.19.0-cp312-cp312-manylinux_2_31_x86_64.whl.metadata (4.3 kB)
  Using cached vtk-9.5.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.6 kB)
  Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached dash-3.2.0-py3-none-any.whl.metadata (10 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached configargparse-1.7.1-py3-none-any.whl.metadata (24 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached pillow-12.0.0-cp312-cp312-manylinux_2_27

In [2]:
from genericpath import exists
import numpy as np
import vedo
import trimesh
import open3d as o3d
import random
import os

# Crear directorio para los archivos de salida
output_dir = "output_3d"
os.makedirs(output_dir, exist_ok=True)

# Crear lista de coordenadas para puntos en el espacio 3D
def generar_puntos(num_puntos=10, rango=5):
    """Genera puntos aleatorios en el espacio 3D"""
    puntos = []
    for _ in range(num_puntos):
        x = random.uniform(-rango, rango)
        y = random.uniform(-rango, rango)
        z = random.uniform(-rango, rango)
        puntos.append([x, y, z])
    return np.array(puntos)

# Generar primitivas con vedo
def crear_primitivas_vedo(puntos):
    """Crea primitivas 3D usando vedo basado en los puntos proporcionados"""
    objetos = []
    colores = vedo.colors.cmaps_names

    for i, punto in enumerate(puntos):
        # Aplicar condicionales para variar parámetros
        if i % 3 == 0:  # Cada tercer punto es una esfera
            radio = 0.2 + (i * 0.05)  # Tamaño variable
            esfera = vedo.Sphere(pos=punto, r=radio, c=random.choice(colores))
            objetos.append(esfera)
        elif i % 3 == 1:  # Cada tercer punto + 1 es un cubo
            lado = 0.3 + (i * 0.04)  # Tamaño variable
            cubo = vedo.Cube(pos=punto, side=lado, c=random.choice(colores))
            objetos.append(cubo)
        else:  # Cada tercer punto + 2 es un cilindro
            radio = 0.15 + (i * 0.03)
            altura = 0.5 + (i * 0.1)
            # Orientación aleatoria para el cilindro
            punto_final = punto + np.array([random.uniform(-1,1), random.uniform(-1,1), altura])
            cilindro = vedo.Cylinder(pos=[punto, punto_final], r=radio, c=random.choice(colores))
            objetos.append(cilindro)

    return objetos

def mostrar_escena_vedo(objetos):
    """Muestra la escena 3D con vedo"""
    escena = vedo.Plotter()
    for obj in objetos:
        escena.add(obj)
    escena.show(axes=1)  # Mostrar ejes de coordenadas

def exportar_con_vedo(objetos, formato="obj"):
    """Exporta objetos usando vedo"""
    # Contador basado en archivos existentes para evitar sobrescritura
    contador_obj = len([f for f in os.listdir(output_dir) if f.startswith("escena_vedo_") and f.endswith(f".{formato}")])
    # Crear una malla combinada de todos los objetos
    malla_combinada = vedo.merge(objetos)
    # Exportar en el formato especificado
    archivo_salida = f"{output_dir}/escena_vedo_{contador_obj}.{formato}"
    vedo.write(malla_combinada, archivo_salida)
    print(f"Escena exportada como {archivo_salida}")
    return malla_combinada

def bonus_importar_csv():
    """BONUS: Importar datos desde un archivo CSV para generar primitivas"""
    # Crear un CSV de ejemplo con coordenadas y parámetros
    datos_ejemplo = np.random.rand(15, 6)  # x,y,z,tipo,tamaño,color_idx
    np.savetxt(f"{output_dir}/datos_ejemplo.csv", datos_ejemplo, delimiter=",",
               header="x,y,z,tipo,tamaño,color_idx", comments="")

    # Leer datos
    datos = np.loadtxt(f"{output_dir}/datos_ejemplo.csv", delimiter=",", skiprows=1)

    objetos = []
    for fila in datos:
        x, y, z, tipo, tamaño, color_idx = fila
        punto = [x*10-5, y*10-5, z*10-5]  # Escalar a un rango de -5 a 5
        c = vedo.colors.cmaps_names[int(color_idx * len(vedo.colors.cmaps_names)) % len(vedo.colors.cmaps_names)]

        # Crear primitiva según el tipo
        if tipo < 0.33:  # Esfera
            obj = vedo.Sphere(pos=punto, r=tamaño, c=c)
        elif tipo < 0.66:  # Cubo
            obj = vedo.Cube(pos=punto, side=tamaño, c=c)
        else:  # Cilindro
            punto_final = punto + np.array([0, 0, tamaño*2])
            obj = vedo.Cylinder(pos=[punto, punto_final], r=tamaño/2, c=c)

        objetos.append(obj)

    return objetos

def main():
    print("Generando escena 3D con datos aleatorios...")

    # Generar puntos aleatorios
    puntos = generar_puntos(num_puntos=15)
    print(f"Puntos generados: {len(puntos)}")

    # Crear primitivas 3D
    objetos = crear_primitivas_vedo(puntos)
    print(f"Objetos creados: {len(objetos)}")

    # Mostrar escena
    mostrar_escena_vedo(objetos)

    # Exportar
    malla_vedo = exportar_con_vedo(objetos, formato="obj")

    # BONUS: Importar desde CSV
    print("\nGenerando objetos desde archivo CSV...")
    objetos_csv = bonus_importar_csv()
    mostrar_escena_vedo(objetos_csv)
    malla_csv = exportar_con_vedo(objetos_csv, formato="obj")

    print("\n¡Proceso completado! Los archivos están disponibles en la carpeta:", output_dir)

# Ejecutar el script principal
if __name__ == "__main__":
    main()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Generando escena 3D con datos aleatorios...
Puntos generados: 15
Objetos creados: 15
Escena exportada como output_3d/escena_vedo_0.obj

Generando objetos desde archivo CSV...
Escena exportada como output_3d/escena_vedo_1.obj

¡Proceso completado! Los archivos están disponibles en la carpeta: output_3d
